In [1]:
import pandas as pd
import matplotlib.pyplot as plt          # plotting
import numpy as np                       # dense matrices
from scipy.sparse import csr_matrix      # sparse matrices
%matplotlib inline

### Load in the dataset

We will be using the same dataset of Wikipedia pages that we used in the Machine Learning Foundations course (Course 1). Each element of the dataset consists of a link to the wikipedia article, the name of the person, and the text of the article (in lowercase). 

In [24]:
wiki = pd.read_csv('people_wiki.csv')
wiki.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


### Extract word count vectors

For your convenience, we extracted the word count vectors from the dataset. The vectors are packaged in a sparse matrix, where the i-th row gives the word count vectors for the i-th document. Each column corresponds to a unique word appearing in the dataset. The mapping between words and integer indices are given in people_wiki_map_index_to_word.gl.

To load in the word count vectors, define the function

In [4]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

In [5]:
word_count = load_sparse_csr('people_wiki_word_count.npz')

In [6]:
tf_idf = load_sparse_csr('people_wiki_tf_idf.npz')

In [7]:
import json

map_index_to_word = json.loads(open('people_wiki_map_index_to_word.json').read())

### Find nearest neighbors using word count vectors

Let's start by finding the nearest neighbors of the Barack Obama page using the word count vectors to represent the articles and Euclidean distance to measure distance. For this, we will use scikit-learn's implementation of k-nearest neighbors. We first create an instance of the NearestNeighbor class, specifying the model parameters. Then we call the fit() method to attach the training set.



In [8]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='euclidean', algorithm='brute')
model.fit(word_count)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='euclidean',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

Run the following cell to obtain the row number for Obama's article:

In [10]:
wiki[wiki['name'] == 'Barack Obama']

,URI,name,text
35817,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...


Let us run the k-nearest neighbor algorithm with Obama's article. Since the NearestNeighbor class expects a vector, we pass the 35817th row of word_count vector.

In [11]:
distances, indices = model.kneighbors(word_count[35817], n_neighbors=10) # 1st arg: word count vector

The query returns the indices of and distances to the 10 nearest neighbors. To display the indices and distances together with the article name, run

In [95]:
neighbors = pd.DataFrame({'distance':distances.flatten(), 'index':indices.flatten()})

distance_obama = wiki.join(neighbors.set_index('index')).fillna(1000).sort_values(by = ['distance'], ascending = True)[['name', 'distance']].iloc[:10]
distance_obama

,name,distance
35817,Barack Obama,0.000000
24478,Joe Biden,33.075671
28447,George W. Bush,34.394767
35357,Lawrence Summers,36.152455
14754,Mitt Romney,36.166283
13229,Francisco Barrio,36.331804
31423,Walter Mondale,36.400549
22745,Wynn Normington Hugh-Jones,36.496575
36364,Don Bonker,36.633318
9210,Andy Anstett,36.959437


### Interpreting the nearest neighbors

All of the 10 people are politicians, but about half of them have rather tenuous connections with Obama, other than the fact that they are politicians.

 *   Francisco Barrio is a Mexican politician, and a former governor of Chihuahua.
 *   Walter Mondale and Don Bonker are Democrats who made their career in late 1970s.
 *   Wynn Normington Hugh-Jones is a former British diplomat and Liberal Party official.
 *   Andy Anstett is a former politician in Manitoba, Canada.

Nearest neighbors with raw word counts got some things right, showing all politicians in the query result, but missed finer and important details.

For instance, let's find out why Francisco Barrio was considered a close neighbor of Obama. To do this, let's look at the most frequently used words in each of Barack Obama and Francisco Barrio's pages.

First, run the following cell to obtain the word_count column, which represents the word count vectors in the dictionary form. This way, we can quickly recognize words of great importance.

Note. Understanding this code is not required for completing this assignment. Feel free to treat it as a black box.

In [35]:
def unpack_dict(matrix, map_index_to_word):
    #table = list(map_index_to_word.sort('index')['category'])
    # if you're not using SFrame, replace this line with
    table = sorted(map_index_to_word, key=map_index_to_word.get)
    
    
    data = matrix.data
    indices = matrix.indices
    indptr = matrix.indptr
    
    num_doc = matrix.shape[0]

    return [{k:v for k,v in zip([table[word_id] for word_id in indices[indptr[i]:indptr[i+1]] ],
                                 data[indptr[i]:indptr[i+1]].tolist())} \
               for i in range(num_doc) ]

wiki['word_count'] = unpack_dict(word_count, map_index_to_word)

In [36]:
wiki.head()

,URI,name,text,word_count
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{'brisbaneafter': 1, 'edflhe': 1, 'aflfrom': 1..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{'maladaptation': 1, 'phasedelay': 1, '25hour'..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{'germanyover': 1, 'bluesgospel': 1, 'harpdog'..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{'fantasticrottensteiner': 1, 'waidmannsfeld':..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{'arhm': 3, 'gangstergenka': 1, 'kuhnja': 1, '..."


To make things even easier, we provide a utility function that displays a dictionary in tabular form:

In [79]:
import operator

def word_count_sort(word_dict):
    word_dict_sorted = sorted(word_dict.items(),key=operator.itemgetter(1),reverse=True)
    return word_dict_sorted


def top_words(name):
    """
    Get a table of the most frequent words in the given person's wikipedia page.
    """
    row = wiki[wiki['name'] == name]
    word_count_table = row['word_count'].apply(word_count_sort).iloc[0]
    return pd.DataFrame(word_count_table, columns = ['word', 'count'])

obama_words = top_words('Barack Obama')
obama_words.head()

,word,count
0,the,40
1,in,30
2,and,21
3,of,18
4,to,14


In [80]:
barrio_words = top_words('Francisco Barrio')
barrio_words.head()

,word,count
0,the,36
1,of,24
2,and,18
3,in,17
4,he,10


Let's extract the list of most frequent words that appear in both Obama's and Barrio's documents. We've so far sorted all words from Obama and Barrio's articles by their word frequencies. We will now use a dataframe operation known as merge. 

In [86]:
combined_words = obama_words.merge(barrio_words, on = 'word')
combined_words.head()

,word,count_x,count_y
0,the,40,36
1,in,30,17
2,and,21,18
3,of,18,24
4,to,14,9


In [89]:
combined_words = combined_words.rename(columns={'count_x':'Obama', 'count_y':'Barrio'})
combined_words.head()

,word,Obama,Barrio
0,the,40,36
1,in,30,17
2,and,21,18
3,of,18,24
4,to,14,9


# Question 1
Among the words that appear in both Barack Obama and Francisco Barrio, take the 5 that appear most frequently in Obama. How many of the articles in the Wikipedia dataset contain all of those 5 words?

Hint:

  *  Refer to the previous paragraph for finding the words that appear in both articles. Sort the common words by their frequencies in Obama's article and take the largest five.
  *  Each word count vector is a Python dictionary. For each word count vector in SFrame, you'd have to check if the set of the 5 common words is a subset of the keys of the word count vector. Complete the function has_top_words to accomplish the task.
  *  Convert the list of top 5 words into set using the syntax "set(common_words)", where common_words is a Python list. See this link if you're curious about Python sets.
  *  Extract the list of keys of the word count dictionary by calling the keys() method.
  *  Convert the list of keys into a set as well.
  *  Use issubset() method to check if all 5 words are among the keys.
  *  Now apply the has_top_words function on every row of the SFrame.
  *  Compute the sum of the result column to obtain the number of articles containing all the 5 top words. 

In [92]:
common_words = combined_words['word'][0:5]

def has_top_words(word_count_vector):
    # extract the keys of word_count_vector and convert it to a set
    unique_words = set(word_count_vector.keys())
    # return True if common_words is a subset of unique_words
    # return False otherwise
    return set(common_words).issubset(unique_words)

wiki['has_top_words'] = wiki['word_count'].apply(has_top_words)

# use has_top_words column to answer the quiz question
wiki['has_top_words'].sum()

56066

In [94]:
wiki.head()

,URI,name,text,word_count,has_top_words
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{'brisbaneafter': 1, 'edflhe': 1, 'aflfrom': 1...",True
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{'maladaptation': 1, 'phasedelay': 1, '25hour'...",True
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{'germanyover': 1, 'bluesgospel': 1, 'harpdog'...",True
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{'fantasticrottensteiner': 1, 'waidmannsfeld':...",True
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{'arhm': 3, 'gangstergenka': 1, 'kuhnja': 1, '...",False


# Question 2
Measure the pairwise distance between the Wikipedia pages of Barack Obama, George W. Bush, and Joe Biden. Which of the three pairs has the smallest distance?

In [106]:
from sklearn.metrics.pairwise import pairwise_distances

def pairwise_distance_calculation(people1, people2):
    return pairwise_distances(word_count[people1.index], word_count[people2.index], metric = 'euclidean')[0][0]

obama = wiki[wiki['name'] == 'Barack Obama']
bush = wiki[wiki['name'] == 'George W. Bush']
biden = wiki[wiki['name'] == 'Joe Biden']

print('pairwise distance between Barack Obama and George W. Bush is: ', pairwise_distance_calculation(obama, bush))
print('pairwise distance between Barack Obama and Joe Biden is:      ', pairwise_distance_calculation(obama, biden))
print('pairwise distance between George W. Bush and Joe Biden is:    ', pairwise_distance_calculation(bush, biden))

pairwise distance between Barack Obama and George W. Bush is:  34.3947670438
pairwise distance between Barack Obama and Joe Biden is:       33.0756708171
pairwise distance between George W. Bush and Joe Biden is:     32.7566787083


# Question 3
Collect all words that appear both in Barack Obama and George W. Bush pages. Out of those words, find the 10 words that show up most often in Obama's page. Which of the following is NOT one of the 10 words?

In [108]:
bush_words = top_words('George W. Bush')
combined_words_bush_obama = obama_words.merge(obama_words, on = 'word').rename(columns = {'count_x':'Obama', 'count_y':'Bush'})
combined_words_bush_obama.head(10)

,word,Obama,Bush
0,the,40,40
1,in,30,30
2,and,21,21
3,of,18,18
4,to,14,14
5,his,11,11
6,obama,9,9
7,act,8,8
8,he,7,7
9,a,7,7


### Extract the TF-IDF vectors

Much of the perceived commonalities between Obama and Barrio were due to occurrences of extremely frequent words, such as "the", "and", and "his". So the nearest neighbors algorithm is recommending plausible results sometimes for the wrong reasons.

To retrieve articles that are more relevant, we should focus more on rare words that don't happen in every article. TF-IDF (term frequency–inverse document frequency) is a feature representation that penalizes words that are too common. Let us load in the TF-IDF vectors and repeat the nearest neighbor search.

For your convenience, we extracted the TF-IDF vectors from the dataset. The vectors are packaged in a sparse matrix, where the i-th row gives the TF-IDF vectors for the i-th document. Each column corresponds to a unique word appearing in the dataset. The mapping between words and integer indices are given in people_wiki_map_index_to_word.gl.

To load in the TF-IDF vectors, run

In [109]:
tf_idf = load_sparse_csr('people_wiki_tf_idf.npz')

In addition to the sparse matrix, we also store the TF-IDF vectors in dictionary form as well, to allow for easy interpretation.

In [110]:
wiki['tf_idf'] = unpack_dict(tf_idf, map_index_to_word)

### Find nearest neighbors using TF-IDF vectors

Since we are now using a different set of features, we should create a new nearest neighbor model. Create another instance of the NearestNeighbor class as follows. Then call the fit() method to associate it with the TF-IDF vectors.

In [111]:
model_tf_idf = NearestNeighbors(metric='euclidean', algorithm='brute')
model_tf_idf.fit(tf_idf)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='euclidean',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

Perform the nearest neighbor search by running

In [112]:
distances, indices = model_tf_idf.kneighbors(tf_idf[35817], n_neighbors=10)

In [113]:
neighbors = pd.DataFrame({'distance':distances.flatten(), 'index':indices.flatten()})

distance_obama = wiki.join(neighbors.set_index('index')).fillna(1000).sort_values(by = ['distance'], ascending = True)[['name', 'distance']].iloc[:10]
distance_obama

,name,distance
35817,Barack Obama,0.000000
7914,Phil Schiliro,106.861014
46811,Jeff Sessions,108.871674
44681,Jesse Lee (politician),109.045698
38376,Samantha Power,109.108106
6507,Bob Menendez,109.781867
38714,Eric Stern (politician),109.957788
44825,James A. Guest,110.413889
44368,Roland Grossenbacher,110.470609
33417,Tulsi Gabbard,110.696998


Let's determine whether this list makes sense.

  *  With a notable exception of Roland Grossenbacher, the other 8 are all American politicians who are contemporaries of Barack Obama.
  *  Phil Schiliro, Jesse Lee, Samantha Power, and Eric Stern worked for Obama.

Clearly, the results are more plausible with the use of TF-IDF. Let's take a look at the word vector for Obama and Schilirio's pages. Notice that TF-IDF representation assigns a weight to each word. This weight captures relative importance of that word in the document. Let us sort the words in Obama's article by their TF-IDF weights; we do the same for Schiliro's article as well.

In [118]:
def top_words_tf_idf(name):
    row = wiki[wiki['name'] == name]
    word_count_table = row['tf_idf'].apply(word_count_sort).iloc[0]
    return pd.DataFrame(word_count_table, columns = ['word', 'count'])

obama_tf_idf = top_words_tf_idf('Barack Obama')
obama_tf_idf.head()

,word,count
0,obama,43.295653
1,act,27.678223
2,iraq,17.747379
3,control,14.887061
4,law,14.722936


In [119]:
schiliro_tf_idf = top_words_tf_idf('Phil Schiliro')
schiliro_tf_idf.head()

,word,count
0,schiliro,21.972991
1,staff,15.856442
2,congressional,13.547088
3,daschleschiliro,10.986495
4,obama,9.621256


In [122]:
combined_words_tf_idf = obama_tf_idf.merge(schiliro_tf_idf, on = 'word').rename(columns = {'count_x':'Obama', 'count_y':'Schiliro'})
combined_words_tf_idf.head(10)

,word,Obama,Schiliro
0,obama,43.295653,9.621256
1,law,14.722936,7.361468
2,democratic,12.410689,6.205344
3,senate,10.164288,3.388096
4,presidential,7.386955,3.693478
5,president,7.226869,9.033587
6,policy,6.095386,3.047693
7,states,5.473201,1.824400
8,office,5.248173,2.624086
9,2011,5.107041,3.404694


# Question 4
Among the words that appear in both Barack Obama and Phil Schiliro, take the 5 that have largest weights in Obama. How many of the articles in the Wikipedia dataset contain all of those 5 words?

In [123]:
common_words_tf_idf = combined_words_tf_idf['word'][0:5]

def has_top_words_tf_idf(tf_idf_vector):
    # extract the keys of word_count_vector and convert it to a set
    unique_words = set(tf_idf_vector.keys())
    # return True if common_words is a subset of unique_words
    # return False otherwise
    return set(common_words_tf_idf).issubset(unique_words)

wiki['has_top_words_tf_idf'] = wiki['tf_idf'].apply(has_top_words_tf_idf)

# use has_top_words column to answer the quiz question
wiki['has_top_words_tf_idf'].sum()

14

# Question 5
Compute the Euclidean distance between TF-IDF features of Obama and Biden. Round your answer to 3 decimal places. Use American-style decimals (e.g. 110.921).

In [125]:
def pairwise_distance_calculation_tf_idf(people1, people2):
    return pairwise_distances(tf_idf[people1.index], tf_idf[people2.index], metric = 'euclidean')[0][0]

print('pairwise distance between Barack Obama and Joe Biden is: ', pairwise_distance_calculation_tf_idf(obama, biden))


pairwise distance between Barack Obama and Joe Biden is:  123.29745601


 The distance is larger than the distances we found for the 10 nearest neighbors. But one may wonder, is Biden's article that different from Obama's, more so than, say, Schiliro's? It turns out that, when we compute nearest neighbors using the Euclidean distances, we unwittingly favor short articles over long ones. Let us compute the length of each Wikipedia document, and examine the document lengths for the 100 nearest neighbors to Obama's page.

In [132]:
# Comptue length of all documents
def compute_length(row):
    return len(row.split())
wiki['length'] = wiki['text'].apply(compute_length)


In [133]:
# Compute 100 nearest neighbors and display their lengths
distances, indices = model_tf_idf.kneighbors(tf_idf[35817], n_neighbors=100)
neighbors = pd.DataFrame({'distance':distances.flatten(), 'index':indices.flatten()})

nearest_neighbors_euclidean = wiki.join(neighbors.set_index('index')).fillna(1000).sort_values(by = ['distance'], ascending = True)[['name', 'distance', 'length']].iloc[:10]

print(nearest_neighbors_euclidean)

                          name    distance  length
35817             Barack Obama    0.000000     540
7914             Phil Schiliro  106.861014     208
46811            Jeff Sessions  108.871674     230
44681   Jesse Lee (politician)  109.045698     216
38376           Samantha Power  109.108106     310
6507              Bob Menendez  109.781867     220
38714  Eric Stern (politician)  109.957788     255
44825           James A. Guest  110.413889     215
44368     Roland Grossenbacher  110.470609     201
33417            Tulsi Gabbard  110.696998     228
